# Data Cleaning Function
#### def data_cleaning (dataframe,replace_missing_value)
#### return dataframe
##### Written by : Loh Khai Shyang
##### Scripted Date : 22 Nov 2021

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def data_cleaning(df,replace_missing_value = value to be filled into missing rows):
    ### 1. search_remove_individual_value_features
    ### 2. replace_missing_data
    ### 3. blank_space_repalcement
    
    ### return dataframe thru data cleaning process
    
    ### Hyper- Parameter
        # 1. df = Dataframe to be clean
        # 2. replace_missing_value = value to be filled into missing rows

        
    def basic_data_info(df):
        # check dataset overall information
        print('\nData Cleaning Report - Basic Data Informations\n')
        print(df.info())

        print('\nData Cleaning Report - Summary of total "NAN" rows\n')
        # identify amount of  "NAN" row
        print(df.isna().sum())



    def search_remove_individual_value_features(df):
        # remove features if :
            # X[i].unique() == 1
            # X[i].unique() == 2 & np.nan isin( X[i].unique() )
            # X[i].unique() == 2 & ' ' isin( X[i].unique())  ### ' ' == empty space
        
        column_name = df.columns
        remove_single_value_features_list=[]
        
        for name in column_name:


            if len(df[name].unique())==1: # X[i].unique() == 1
                remove_single_value_features_list=np.append(name,remove_single_value_features_list)
            
            elif len(df[name].unique())==2 and np.nan in(df[name].unique()): # X[i].unique() == 2 & np.nan isin( X[i].unique() )
                remove_single_value_features_list=np.append(name,remove_single_value_features_list)
                
            elif len(df[name].unique())==2 and ' ' in(df[name].unique()): # X[i].unique() == 2 & ' ' isin( X[i].unique())  ### ' ' == empty space
                remove_single_value_features_list=np.append(name,remove_single_value_features_list)


        if len(remove_single_value_features_list)>=1:
            print('\nData Cleaning Report - Individual Value feature removed : ',remove_single_value_features_list)
            df.drop(remove_single_value_features_list,axis=1,inplace=True)
        else:
            print("\nData Cleaning Report - No Individual Value feature Found !!\n")

        return df


    def replace_missing_data(df,replace_missing_value):
        # user can define what value to fill in to missing data
        # "nan" and "blank space" consider missing data

        column_list= df.columns
        missing_EmptySpace_data_dict_list={} # Store "Empty Space" row in dictionary by column name
        missing_NAN_data_dict_list={} # # Store "nan" row in dictionary by column name

        for column in column_list:
            if len(df.loc[df[column]==' '])!= 0 :
                missing_EmptySpace_data_dict_list[column]=df.loc[df[column]==' '].index # index 
            elif len(df.loc[df[column]==np.nan])!=0:
                missing_NAN_data_dict_list[column]=df.loc[df[column]==np.nan].index

        ## replace missing data function
        if len(missing_EmptySpace_data_dict_list)!=0 : # check empty_space_column dictionary is empty or not

            print('\nData Cleaning Report - Have Missing "Empty Space" Data :\n',missing_EmptySpace_data_dict_list)

            for key in missing_EmptySpace_data_dict_list.keys():
                df.loc[missing_EmptySpace_data_dict_list[key],key]=replace_missing_value # df.loc[idx list, column name] = 0

        elif len(missing_NAN_data_dict_list)!=0 : # check empty_space_column dictionary is empty or not:

            print('\nData Cleaning Report - Have Missing "NaN" Data :\n',missing_NAN_data_dict_list)

            for key in missing_NAN_data_dict_list.keys():
                df.loc[missing_NAN_data_dict_list[key],key]=replace_missing_value # df.loc[idx list, column name] = 0

        else:
            print('\nData Cleaning Report - No Missing Data or "Nan" row Found !!\n')

        return df


    def blank_space_repalcement(df):
        # Dataframe Column Name Blank Space Replacement
        df.columns=df.columns.str.replace(' ','_')
        df.replace(' ','_',regex=True,inplace=True)

        return df

    basic_data_info(df)
    df=search_remove_individual_value_features(df)
    df=replace_missing_data(df,replace_missing_value)
    df=blank_space_repalcement(df)

    return df

In [ ]:
def imbalance_data_check(df,label_name):
    # SuitabLe for categorical class label [both ordinal or nominal data]
    
    
    
    ## Frequency Table of Label data ##
#     print('\n Imbalance Data Check - Label vs Features Table \n',df.groupby([label_name]).count(),'\n')
    
    freq_table=df.groupby([label_name]).size().reset_index(name='Count')

    print('\n Imbalance Data Check - Frequency Table of Label Data :\n',freq_table,'\n')
    
    
    
    ## % of label's class distribution data Summary ##
    summary={}

    unique_class=freq_table[label_name].unique()
    total_count=freq_table['Count'].sum()
    
    for i in range(len(unique_class)):
        summary[unique_class[i]]= [(freq_table['Count'][i]/total_count)*100] 
        
    summary_df=pd.DataFrame(data=summary)
    
    print(f'Imbalance Data Check - Label Data Distribution % :\n{summary_df}')

          
          
          
    ##  Plot Figure of Label class data distribution ##
    print(f'\nImbalance Data Check - Data Distribution % Summary Plot :\n') # Imbalance data check summary
    
    sns.set_theme(style="whitegrid")
    ax=sns.barplot(x=label_name,y="Count", data=freq_table)
    
    for i in unique_class:
        num=round(summary[i][0],2)
#         value=str(num)+'%'
        ax.text(i,num,round(num,2), color='black', ha="center")

    return

In [ ]:
def numeric_histogram_plot_for_classification_model(df,label_feature,save_photo):
    # label_feature = input label feature name
    # df = input dataframe to be process
    # save_photo = True ( auto saved), Default=="False"
    
    # sns.displot(daframe,x="column name to plot on x-axis",hue=label_feature, element="step")
    # subplot link : https://datavizpyr.com/seaborn-join-two-plots-with-shared-y-axis/
    
    
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    plot_columns=list(df.select_dtypes(include=['int32','int64','float64','float32']).columns)
    
    if label_feature in(plot_columns):
        plot_columns.remove(label_feature)
    
    for i,column in zip( range(len(plot_columns)), plot_columns ):
        plt.figure(i)
        sns_plot=sns.displot(data=df,x=column,hue=label_feature, element="step",kde=True)
        plt.title(column+" - Displot Plot")
        
#         sns.histplot(df, x=column,hue=label_feature, element="step", kde=True)
#         plt.title(column+" - Histogram Plot")
#         plt.show()
        
        if save_photo==True:
            sns_plot.savefig(column+".png")     # auto Saved figure file 
            
    return


In [ ]:
def numeric_histplot(df,label):
    # label must vbe a list of feature to plot
    
    for i, feature in label:
        plt.figure(i)
        sns_plot=sns.displot(data=df,x=feature, element="step",kde=True)
        plt.title(feature+" - Displot Plot")
        
    return

def numeric_compare_histplot(df_1,df_2,compare_label,compare_name):
    # Plot two dataframe histplot for in one graph df_1 and df_2 for comparison plot
    # compare_label must be a list of feature to plot
    # compare_name = show name of dataframe compare plot legend
    
    from scipy.stats import norm
    
    for i, feature in zip(range(len(compare_label)), compare_label) :
        fig,ax=plt.subplots(figsize=(11.7,8.27)) # figsize = in inch
        sns.distplot(df_1[feature], ax=ax, color='r',kde=True)
        sns.distplot(df_2[feature], ax=ax, color='b', kde=True)
        plt.title(feature+" - Distribution Count Plot")
        plt.legend(title='Distribution', loc='upper left', labels=[compare_name[0],compare_name[1]])
        
        fig,ax1=plt.subplots(figsize=(11.7,8.27))
        sns.distplot(df_1[feature], ax=ax1,color='r', kde=True)
        sns.distplot(df_2[feature], ax=ax1,color='b', kde=True)
        ax1.set_ylim([0,0.5])
        plt.title(feature+" - Distribution Density Plot")
        plt.legend(title='Distribution', loc='upper left', labels=[compare_name[0],compare_name[1]])
        
        plt.show()

        
        
    return

In [ ]:
def z_score_outlier_detection_numeric(df,label,gt_value,numeric_feature_column,outlier_threshold_rate,,feature_remove_decision=False,save_plot=False):
    ### 1. Calcualte z-score of whole df
    ### 2. Detect consider outlier if [- 3sigma >= Z-score >= +3 sigma  == consider outlier ]
    ### 3. Remove outlier column from df if:
            # [ outlier rate > outlier_threshold_rate ]
            # feature_remove_decision = True
    ### 4. Plot Z-score Box plot for each feature column
    ### 5. Plot output features distribution graph 
    
    
    ### Hyper Parameter
    # 1. outlier_density_threshold, % = acceptable z-score density threshold before remove
    # 2. label is used to plot 
    # 3. gt_value= Label Ground Truth value
    # 3. Z_score_outlier_threshold = z_score removal threhold
    # 4. numeric_feature_column = feature to calculate z-score analysis (user need to exclude categorical number column out of the list)
    # 5. outlier_removal_rate = column removal % if outlier rate exceed threshold
    # 6. save_photo = True = Auto saved photo
    
    # return ploted z-score box plot
    # return outlier cleaned dataframe ( clean column outlier of exceed outlier rate threshold)
    
    
    import matplotlib.pyplot as plt
    import seaborn as sns 
    from scipy import stats
    
    ## Dataframe Describe Info ##
    print(label+' - Data Attribute\n',df[label].describe())
    
    
    ### Plot Distrubution Label Data Analysis Graph ###
    stdev=round(df[label].std(),2) # label stdev
    mean=round(df[label].mean(),2) # label mean value 
    min_value=round(min(df[label]),2) # min label value 
    max_value=round(max(df[label]),2) # max label value

    
    UCL= mean+3*stdev # label negative 3-sigma
    LCL = mean-3*stdev # label positive 3-sigma
    
    no_outof_gt_value_spec=len(df[df[label]<gt_value]) # numbers of label value smaller than groundtruth value
    no_within_gt_value_spec=len(df)-len(df[df[label]<gt_value]) # numbers of label value within ground truth value
    
    no_within_3sigma_spec=len(df)-len(df[(df[label]<LCL) | ( df[label]>UCL)]) # numbers of label value within UCL & LCL
    
    ## Plot Label data Histogram ##
    sns.set(rc={'figure.figsize':(20,15)})
    plt.figure(100)
    sns.distplot(df[label],kde=True)

    plt.axvline(gt_value,linestyle='--',color='red') # Numeric label Ground Truth line
    plt.axvline(UCL,linestyle='--',color='blue')  # positive 3-sigma
    plt.axvline(LCL,linestyle='--',color='green')  # negative 3-sigma
    
    plt.title(label+ 'Distribution Plot [Within '+label+' Spec:'+str(no_within_gt_value_spec)+', OutofSpec:'+str(no_outof_gt_value_spec)+'] [ Within UCL/LCL:'+str(no_within_3sigma_spec)+'/'+str(len(df))+' ]',fontsize=12)
    plt.legend(['Dataset N:'+str(len(df))+' Mean:'+str(mean)+' Max:'+str(max_value)+' Min:'+str(min_value),
                label+' Ground Truth',
                'UCL',
                'LCL'],
               fancybox=True,
               framealpha=1,
               shadow=True,
               borderpad=1,fontsize=12)
    
    plt.xlabel(str(label)+' value',fontsize=12)
    plt.ylabel('Counts',fontsize=12)
    
    if save_plot==True:
        plt.savefig("Label_Distribution_plot.png")     # auto Saved figure file

    
    ############################################################################################################################################################
    
    
    
    ## Calcualte Z-score dataframe ##
    df_feature_column=df.columns
    
    new_numeric_feature_column=list() # create a new list to store exist numeric feature column after data cleaning
    
    for i in df_feature_column:
        if i in(numeric_feature_column):
            new_numeric_feature_column.append(i)
        
    df_z_score=df[new_numeric_feature_column] # form new dataframe of new_numeric_feature_column
    
    
    for feature in df_z_score:
        stdev=df_z_score[feature].std()
        mean=df_z_score[feature].mean()
        df_z_score[feature]=(df_z_score[feature]-mean)/stdev
    
    
    
    
    def outlier_removal(df,df_z_score,outlier_threshold_rate,feature_remove_decision,save_plot):
        #  - 3sigma >= Z-score >= +3 sigma  == consider outlier
        
        outlier_rate_dict={}
        total_outlier =0
        
        for column in df_z_score:
            outlier_number =len(df_z_score[df_z_score[column]>3.0]) + len(df_z_score[df_z_score[column]<-3.0])
            outlier_rate=outlier_number/len(df_z_score[column])*100    # outlier rate in %
            outlier_rate_dict[column]=outlier_rate
            
            total_outlier=total_outlier + outlier_number
        
        overall_outlier_rate=total_outlier/ (len(df_z_score)*len(df_z_score.columns))*100 # overall outlier rate of whole dataframe in %
        
        outlier_remove_column=[]
        for key in outlier_rate_dict:
            if outlier_rate_dict[key] > outlier_threshold_rate:
                outlier_remove_column.append(key)
        
        outlier_df=pd.DataFrame(outlier_rate_dict.items(),columns=["Features","Outlier_rate"])
        outlier_df.sort_values(by=["Outlier_rate"], ascending=True, inplace=True)  
        
        if feature_remove_decision = True : # if feature_remove_decision is True it will remove the outlier column from df
            df.drop(outlier_remove_column,axis=1,inplace=True) # Remove train data outlier column
            print('Outlier Remove Decision "Enable" ')
        else:
            print('Outlier Remove Decision "Disable" ')
            
        print("outlier_remove_column[>"+str(outlier_threshold_rate)+"] : ",outlier_remove_column) # show outlier removed column
        
        plt.figure()
        ax1=sns.lineplot(data=outlier_df, x="Features", y="Outlier_rate", marker='o', sort=True)
        ax1.tick_params(axis='x',rotation=90)  # rotate label x by 90 degree
        plt.xlabel('Features')
        plt.ylabel("Outlier Percentage %")
        plt.title("Outlier Rate Plot -- Data Shape:"+str(df_z_score.shape)+"   Overall Outlier Rate[%]:"+str(overall_outlier_rate)+"   Feature Reduction:"+str(len(outlier_remove_column))+"(<"+str(outlier_threshold_rate)+"%)")
        
        if save_plot==True:
            plt.savefig("Outlier_Rate_plot.png")      
        return df
        
    
    

    
    def z_score_boxplot(df,save_plot):
        # input 
        ### Each graph boxplot contains only 20 features

        start_index=0 # plot_column_index start index value 
        final_index=len(df.columns)

        plot_column_list=[]
        plot_column_index=[]

        max_limit_value=max(df.max())
        min_limit_value=min(df.min())

        ### Form index list of column to seperate different graph per box plot ###
        for i in range(0,len(list(df.columns)),20): # seperate 20 features per graph of box plot
            plot_column_index.append(i)
        plot_column_index.remove(0)

        ### Form Column list of each graph of box plot ###
        for i in plot_column_index :
            plot_column_list.append(list(df.columns[start_index:i]))
            start_index=i

        if start_index<final_index:
            plot_column_list.append(list(df.columns[start_index:final_index]))
            
        pic_no=1
        ### Plot box plot ###
        for column in plot_column_list:

            sns.set(rc={'figure.figsize':(20,15)})
            plt.figure()
            sns.boxplot(data=df[column],orient="h",fliersize=6)
            plt.xlim(min_limit_value-2,max_limit_value+2)
            plt.xlabel('Z-Score')
            plt.ylabel("Features")
            plt.title("Z-score vs Features BoxPlot")
            
            ### Save Plot ###
            if save_plot==True:
                plt.savefig("Boxplot_"+str(pic_no)+".png")     # auto Saved figure file
                pic_no+=1 
        return
    

    z_score_boxplot(df_z_score,save_plot) # Call z_score_boxplot function 
    df = outlier_removal(df,df_z_score,outlier_threshold_rate,feature_remove_decision,save_plot)
    
    return 

In [ ]:

def correlation_filter_function(dataset,corr_dataset,label,corr_method,corr_threshold,remove_feature_decision=False,save_figure=False):
    ''' Function will Calculate Correlation Coefficient 
        Plot Correlation Coeffiecent as Heat map
        Show Total Rejected Features ( > corr_threshold )
        Show High correlated features pair and corr value ( > corr_threshold )
        Plot Top 20 Features to Features Correlation Coeficent Heatmap --> label column
        Plot Top 20 Features vs Label Features
        return : Dataframe ( remove or not highly correlated feature )
    '''
    
    """ Function Variables """
    # dataset = original dataframe 
    # corr_dataset: Consist of numeric feature from dataframe only
    # label = Label Feature column
    # corr_method : "pearson", "kendall", "spearman"
    # corr_threshold : threshold to reject "corr value > corr_threshold"
    # remove_feature_decision : True/ False decide to remove or not highly correlated feature from dataframe

    
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    
    
    def _get_diagonal_pairs(corr_dataset):
        
        ''' Get diagonal and lower triangular pairs of correlation matrix '''
        
        pairs_to_drop=set()
        cols=corr_dataset.columns

        for i in range(len(corr_dataset.columns)):
            for j in range(0,i+1):
                pairs_to_drop.add((cols[i],cols[j])) # diagonal label col name [ it store index type of pandas series ]

        return pairs_to_drop


    
    
    def _get_reject_corr_features(corr_dataset,label,corr_method,corr_threshold,top_features_no):
        
        ''' Calculate/Show Features to Features Correlation Coefficient Value '''
        
        corr_df = corr_dataset.corr(method=corr_method).unstack() # corr value non-absolute
        
        corr_abs_df = corr_dataset.corr(method=corr_method).abs().unstack() # absolute corr Series, from "Dataframe" unstack to a list of "Pandas Series"
        labels_to_drop = _get_diagonal_pairs(corr_dataset) 
        
        corr_abs_df = corr_abs_df.drop(labels=labels_to_drop).sort_values(ascending=False) # drop repeated pairs corr value and sort by descending order
        corr_idx = corr_abs_df.index # get pandas series index of descending order
        corr_df=corr_df.reindex(index=corr_idx) # re-arrange without absolute corr value to descending order
        
        print(f'All Correlation Features Pairs (Corr Features Pair Values>{str(corr_threshold)}) :\n\n {corr_df[ corr_df > corr_threshold]}')

        #         corr_df = corr_df.drop(labels=labels_to_drop) # Drop "pandas series" by (column_name_1, column_name_2)
        #         # argsort returns only index of ascending order [::-1] re-arrange index inversly = sort in descending order
        #         sort_corr_abs_idx = corr_abs_df.argsort().[::-1] # return descending sort order index  = Highest Correlation cooeficient index
        
        
        
        
        """ Plot Top Features vs Label Features """
        
        top_highest_corr_features_index=list(corr_dataset.corr()[label].abs().sort_values(ascending=False)[1:top_features_no+1].index) # Sort Corr abs value as descending order and set as list of index

        plt.figure()
        # Heatmap require input 2D
        # plot single column dataframe into heatmap need to df[['columns']] will result in 2D suitable for heatmap input
        sns.heatmap(corr_dataset.corr()[[label]].loc[top_highest_corr_features_index],annot=True, fmt=".2f", cmap='Blues')
        plt.title(f' Top {top_features_no} Features vs {label} {corr_method} Corrleation ')
        
        # Save Figure
        if save_figure == True:
            plt.savefig(f'Top_{top_features_no}_Features_vs_{label}_{corr_method}_Corrleation.png')
              
        
        
        
        """ Plot Top Features to Features Correlation Coeficent Heatmap --> label column """
                
        corr_matrix=corr_dataset[top_highest_corr_features_index].corr()
        plt.figure(figsize=(15,15))
        # "annot" = True = show readings
        # "fmt" = use to set decimal .2f = 2 float decimal
        sns.heatmap(corr_matrix,annot=True, fmt=".2f", cmap='Blues') 
        plt.title(f' Top {top_features_no} {label} - Features to Features {corr_method} Corrleation ')
        
        # Save Figure
        if save_figure == True:
            plt.savefig(f'Top_{top_features_no}_{label}_Features_to_Features_{corr_method}_Corrleation.png')
        
        
        
        """ Correlation Plot and return high correlated features as "col_corr" """
    
        reject_col_corr=set() # set is use because it does not add duplicate same value in a set() variables
        corr_matrix= corr_dataset.corr(method=corr_method)
        
        # Plot figure
        plt.figure(figsize=(12,10))
        sns.heatmap(corr_matrix,annot=True)
        plt.title(f'{corr_method} Correlation Coeficient All Features [Feature Size={corr_matrix.shape}]')
        
        # Save Figure
        if save_figure == True:
            plt.savefig('Correlation_Coefficient_All_Features.png')
        
        # Extract Out of corr_threshold features
        col=corr_matrix.columns 
        
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if (corr_matrix.iloc[i,j])>corr_threshold:
                    
                    # add remove features to set for correlation value of features vs label is the smallest
                    if corr_matrix.loc[col[i],label]< corr_matrix.loc[col[j],label]: 
                        reject_col_corr.add(col[i])
                    else:
                        reject_col_corr.add(col[j]) 
                        
        print(f'\nSize of Rejected Corr features :{len(reject_col_corr)} (>{str(corr_threshold)}) ')
        
        print(f'\n Rejected Corr Features : \n\n {reject_col_corr}')
        
        return reject_col_corr
    
    
    
    
    """ Show Correlation Values of Pair Features " """
    # Run Correlation Coefficient analaysis
    reject_col_corr = _get_reject_corr_features(corr_dataset,label,corr_method,corr_threshold,20)
    
    # Remove out of corr_threshold features
    if remove_feature_decision == True:
        dataset = dataset.drop(reject_col_corr, axis=1)
    
    
    return dataset
    